# generator playground

In [26]:
import numpy as np
import glob
import xarray as xr
from collections.abc import Generator


In [81]:
gen = slice_generator(img_dir='../data/train', slice_size=6, vars_=['t2m'],
                     proc_type='conv_lstm', pixels_x=21, pixels_y=21, debug=True
                     )

In [82]:
in_, out_ = next(gen)

In [85]:
out_['time']

<xarray.DataArray 'time' (time: 6)>
array(['2016-01-01T06:00:00.000000000', '2016-01-01T07:00:00.000000000',
       '2016-01-01T08:00:00.000000000', '2016-01-01T09:00:00.000000000',
       '2016-01-01T10:00:00.000000000', '2016-01-01T11:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2016-01-01T06:00:00 ... 2016-01-01T11:00:00
Attributes:
    long_name:  time

# slice_generator

In [80]:
import numpy as np
import glob
import xarray as xr
from collections.abc import Generator

class slice_generator(Generator):
    """
    A generator that returns a tuple (input_images, output_images), where
    input_images is of shape (1, slice_size, len(vars_), pixels_x, pixels_y)
    output_images is the same shape but starts immediately after end of input
    """
    def __init__(self, img_dir, slice_size, vars_,
                 proc_type, pixels_x, pixels_y, debug):
        self.slice_size = slice_size
        self.vars_ = vars_
        self.proc_type = proc_type
        self.pixels_x = pixels_x
        self.pixels_y = pixels_y
        
        self.netcdf_dirs = sorted(glob.glob(img_dir+"/*.nc"))
        
        self._file_index = 0
        self._counter = 0
        self._debug = debug
        
        self._dataset_current = xr.open_dataset(self.netcdf_dirs[self._file_index])
        self._dataset_current = self._dataset_current[self.vars_]
        
    def throw(self, type=None, value=None, traceback=None):
        raise StopIteration
    
    def send(self, ignored_args):
        input_images = self._get_slices(start=self._counter,
                                  end=self._counter+self.slice_size,
                                  )
        
        # When we get to the end of the file, open up the next file and pull the ouputs from there, 
        # then get ready for the next iteration
        
        if self._counter + 2*self.slice_size > self._dataset_current.sizes['time']:
            self._counter = 0
            self._file_index += 1
            if self._file_index == len(self.netcdf_dirs):
                self._file_index = 0
            
            # Open next .nc file in netcdf_dirs
            self._dataset_current = xr.open_dataset(self.netcdf_dirs[self._file_index])
            self._dataset_current = self._dataset_current[vars_]

            output_images = self._get_slices(start=self._counter,
                                       end=self._counter + self.slice_size, 
                                       )
            self._counter -= self.slice_size
        
        else:
            output_images = self. _get_slices(start=self._counter + self.slice_size,
                                       end=self._counter + 2*self.slice_size,
                                      )
        
        self._counter += self.slice_size
        return (input_images, output_images)

    def _get_slices(self,start:int, end:int):
        if self._debug == True:
            array = self._dataset_current.isel(time=slice(start, end))
            return array
        array = self._dataset_current.isel(time=slice(start, end)).to_array().values
        if self.proc_type == "conv_lstm":
            # switch first and second axes (in practice from (channels, frames, pixels_x, pixels_y) 
            # to (frames, channels, pixels_x, pixels_y) )
            array = np.moveaxis(array, 0, 1)
            # add empty dimension in front for ConvLSTMs
            array.reshape(-1, 1, len(self.vars_), self.pixels_x, self.pixels_y)
            return array
        else:
            raise NameError("proc_type is not recognized. Try 'conv_lstm'")

# Create a class

In [40]:
from collections.abc import Generator

class Fib(Generator):
    def __init__(self):
        self.a, self.b = 0, 1
    def send(self, ignored_arg):
        return_value = self.a
        self.a, self.b = self.b, self.a+self.b
        return return_value
    def throw(self, type=None, value=None, traceback=None):
        raise StopIteration

In [41]:
f = Fib()
for i in range(5):
    print(next(f))

0
1
1
2
3
